In [ ]:
# Could not get this to work.  Exclude from test.

In [ ]:
from utils.const import *
import teehr.classes.duckdb_database as tds
import duckdb

Note: I get a connection error when trying to connect to the DB on s3 via TEEHR

In [ ]:
tdb = tds.DuckDBDatabase(S3_DATABASE_FILEPATH)

In [ ]:
# %%time
# df = tdb.get_metrics(
#     include_metrics=["root_mean_squared_error", "relative_bias", "kling_gupta_efficiency", "r_squared"],
#     group_by=["primary_location_id", "configuration"],
#     order_by=["primary_location_id", "configuration"],
# )
# df

Connecting manually using DuckDB

In [ ]:
S3_DATABASE_FILEPATH

In [ ]:
duckdb.query("LOAD httpfs;")

In [ ]:
duckdb.query("ATTACH 's3://ciroh-rti-public-data/teehr/protocols/science-eval/retro.db' AS retro_db (READ_ONLY);")

In [ ]:
%%time
df = duckdb.query(f"""
WITH joined AS (
        SELECT * FROM retro_db.joined_timeseries
    )
    , metrics AS (
        SELECT
            joined.primary_location_id,joined.configuration
            , 1 - sqrt(
        pow(corr(secondary_value, primary_value) - 1, 2)
        + pow(stddev(secondary_value)
            / stddev(primary_value) - 1, 2)
        + pow(avg(secondary_value) / avg(primary_value) - 1, 2)
    ) as kling_gupta_efficiency
            , sqrt(sum(power(absolute_difference, 2))/count(*))
        as root_mean_squared_error
            , sum(secondary_value - primary_value) / sum(primary_value) AS relative_bias
            , pow(corr(secondary_value, primary_value), 2) as r_squared
        FROM
            joined
        GROUP BY
            joined.primary_location_id,joined.configuration
    )
    SELECT
        metrics.*
    FROM metrics
    ORDER BY
        metrics.primary_location_id,metrics.configuration
;
""").to_df()
df

In [ ]:
duckdb.query("DESCRIBE SELECT * FROM retro_db.joined_timeseries;")